<a href="https://colab.research.google.com/github/IS737StockPicker/stonks/blob/main/IS737_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*#Stock Price Sentimental Analysis Project
##Team Members: Philip Borozenets, Cameron Feder, Kariem Osman & Brian Zuniga


##Overview
(Project Discription)

## Data


This is a combined dataset of new articles September 13, 2022 - March 13, 2023 which was taken from Huffington Post, New York Times, The Guardian & Fin Viz that was used to run sentimental analysis to predict impact on stock prices pulled from yahoo finance to identify a corralation. The `(Dataset name)` Dataset contains X rows and y columns.

# **GitHub Project**
Our team has made a github project, a repository. The URL of the project is (insert URL)

In [ ]:
https://github.com/IS737StockPicker/stonks

##Importing Nessecary Packages & Data in Data Frame:

In [ ]:
#importing packages 
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer #text data
from sklearn.naive_bayes import MultinomialNB # The Naive Bayes algo


from tqdm import tqdm # progress slider for ``for'' loops

#import CSV from Github Repository
#df = pd.read_csv(git_hub_URL);

##Importing Huffington Post News Category Dataset


In [ ]:
huff_news = pd.read_json('/content/drive/MyDrive/News_Category_Dataset_v3.json', lines = True);
huff_news.head ()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [ ]:
#Concise summary of data
huff_news.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [ ]:
# Checking for missing values
huff_news.isna().sum()
#data set has no missing values

link                 0
headline             0
category             0
short_description    0
authors              0
date                 0
dtype: int64

In [ ]:
#Filtering Data to eliminate entries before 09/13/22
huff_news_filtered = huff_news[huff_news['date'] >= pd.to_datetime ('2022-09-13')]

#Droping Unwanted Columns: 
unwanted_columns =['link', 'short_description', 'authors']
huff_news_condensed = huff_news_filtered.drop (unwanted_columns, axis = 1)
 
# Print the new dataframe
huff_news_condensed.head ()

,headline,category,date
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,2022-09-23
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,2022-09-23
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,2022-09-23
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,2022-09-23
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,2022-09-22


In [ ]:
huff_news_condensed['category'].unique ()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT'], dtype=object)

In [ ]:
# Eliminate non-desired topics
huff_news_sentiment = pd.DataFrame ()
desired_topics =['U.S. NEWS','WORLD NEWS','TECH','POLITICS','ENVIRONMENT']
for topics in desired_topics:
  temp = huff_news_condensed[huff_news_condensed['category'] == topics]
  data = [huff_news_sentiment, temp]
  huff_news_sentiment = pd.concat(data)
huff_news_final = huff_news_sentiment.reset_index (drop = True)
huff_news_final

,headline,category,date
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,2022-09-23
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,2022-09-23
2,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,2022-09-22
3,Cleaner Was Dead In Belk Bathroom For 4 Days B...,U.S. NEWS,2022-09-22
4,Reporter Gets Adorable Surprise From Her Boyfr...,U.S. NEWS,2022-09-22
5,Virginia Thomas Agrees To Interview With Jan. ...,U.S. NEWS,2022-09-21
6,One Dead After Plane Crashes At Reno Air Race,U.S. NEWS,2022-09-19
7,Family Of American Held In Afghanistan Says He...,U.S. NEWS,2022-09-19
8,Watchdog Report Finds Some Census Takers Who F...,U.S. NEWS,2022-09-18
9,Alabama Sidesteps Compensation For Survivor Of...,U.S. NEWS,2022-09-16


In [ ]:
#Running News Sentiment Analysis

#Importing nessecary tools
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

#Initalizing Sentiment Analysis 
sia = SIA ()

#Using Sentiment Analysis 
headline_sentiment = []
headline = huff_news_final['headline'].tolist ()
for news in headline:
  pol_score =sia.polarity_scores (news)
  pol_score ['headline']= news 
  headline_sentiment.append (pol_score)
headline_sentiment_df=pd.DataFrame.from_records(headline_sentiment)
headline_sentiment_df
huff_news_final['News Sentiment'] = headline_sentiment_df ['compound']
huff_news_final

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,headline,category,date,News Sentiment
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,2022-09-23,0.0000
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,2022-09-23,-0.5859
2,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,2022-09-22,-0.4939
3,Cleaner Was Dead In Belk Bathroom For 4 Days B...,U.S. NEWS,2022-09-22,-0.5574
4,Reporter Gets Adorable Surprise From Her Boyfr...,U.S. NEWS,2022-09-22,0.6486
5,Virginia Thomas Agrees To Interview With Jan. ...,U.S. NEWS,2022-09-21,0.2023
6,One Dead After Plane Crashes At Reno Air Race,U.S. NEWS,2022-09-19,-0.6486
7,Family Of American Held In Afghanistan Says He...,U.S. NEWS,2022-09-19,-0.2023
8,Watchdog Report Finds Some Census Takers Who F...,U.S. NEWS,2022-09-18,-0.5574
9,Alabama Sidesteps Compensation For Survivor Of...,U.S. NEWS,2022-09-16,0.3612


##New York Times

###Data Collection

In [ ]:
import requests
import os
import json

apikey = os.getenv('NYTIMES_APIKEY', 'SqzyHe7mmHI7o9uARoryVwi8wCVHdKzJ')

ny_data = {}
year = 2022
month = 9

#create for loop for differnet months
filename = f"ny_stories.json"
query_url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apikey}"

r = requests.get(query_url)

ny_data.update(r.json())

# Save NY Data
with open(filename, 'w') as f:
  json.dump(ny_data, f)


###Data Cleaning

In [ ]:
from datetime import datetime
import pandas as pd

#load NY data
# ny_data = json.load(open(filename))

ny_docs = ny_data['response']['docs']
cleaned_ny_data = {
    'headline':[],
    'abstract':[],
    'pub_date': [],
    'section': []
}

start_date = datetime.strptime('2022-09-13', "%Y-%m-%d").date()
end_date = datetime.strptime('2023-03-13', "%Y-%m-%d").date()

for doc in ny_docs:

  #filter sections
  sec_name = doc['section_name']
  if sec_name not in ['Business Day', 'Technology', 'Real Estate']: # 'U.S.' ,''World'
    continue

  #Filter dates
  pub_date = doc['pub_date']
  date = datetime.strptime(pub_date.split('T')[0], "%Y-%m-%d").date()
  if date < start_date or date > end_date:
    continue

  # fill in Dict
  cleaned_ny_data['headline'].append(doc['headline']['main'])
  cleaned_ny_data['abstract'].append(doc['abstract'])
  cleaned_ny_data['pub_date'].append(str(date))
  cleaned_ny_data['section'].append(doc['section_name'])

cleaned_ny_data_df = pd.DataFrame(cleaned_ny_data)
cleaned_ny_data_df.head(10)


,headline,abstract,pub_date,section
0,A $100 Million Bet on Finding the Next ‘Mr. Be...,"Reed Duchscher, the manager of the YouTube meg...",2022-09-13,Business Day
1,Why You Should Decorate Your Hallway (and How ...,"When you’re furnishing your home, it’s easy to...",2022-09-13,Real Estate
2,Markets Plunge as Inflation Data Undercuts Wal...,"Stocks plunged, government bond yields soared ...",2022-09-13,Business Day
3,"Inflation Remained Stubbornly High in August, ...",Overall inflation moderated less than anticipa...,2022-09-13,Business Day
4,"Inflation Explained: The Good, the Bad and the...",Amid hopeful signs that U.S. inflation was abo...,2022-09-13,Business Day
5,U.S. Gas Prices Have Fallen for 91 Straight Da...,The steady decline is also a welcome developme...,2022-09-13,Business Day
6,Twitter Whistle-blower Is Set to Testify in Wa...,Elon Musk’s legal team will follow closely the...,2022-09-13,Business Day
7,‘The Future of Hospitals’: Flexible Space for ...,After struggling to respond to a crushing Covi...,2022-09-13,Business Day
8,Pandemic Aid Cut U.S. Poverty to New Low in 20...,A measure that accounts for all federal subsid...,2022-09-13,Business Day
9,How Is the Economy Doing?,"Most of the economy appears to be doing well, ...",2022-09-13,Business Day


# Stock Ticker Data

In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotnine import *
import yfinance as yf


# IMPORT THE LIBRARY
from datetime import datetime

# CREATE TICKER INSTANCE FOR AMAZON
end_date = datetime.now().strftime('%Y-%m-%d')
stocks = ["AMZN","GOOG","MSFT","GME","TSLA","SPY"]
#stocks =['VENAX']
end_date = datetime.now().strftime('%Y-%m-%d')
i=0
test = pd.DataFrame()
for stock in stocks:
  

  stocks[i] = yf.Ticker(stock)
  stocks[i]= stocks[i].history(start='2022-01-01',end=end_date)
  ticker = stock
  #print(stocks[i])
  stocks[i]['Ticker'] = ticker
  #print(stocks[i])
  #print(stocks[0])
  test = test.append(stocks[i])
  i+=1
test['win_loss_value'] = test.Close - test.Open
test['win_loss'] = np.where(test['win_loss_value']>0, 'Up', 'Down')
test.reset_index(inplace=True)
test.info()
test.Ticker()

<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854 entries, 0 to 1853
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype                           
---  ------          --------------  -----                           
 0   Date            1854 non-null   datetime64[ns, America/New_York]
 1   Open            1854 non-null   float64                         
 2   High            1854 non-null   float64                         
 3   Low             1854 non-null   float64                         
 4   Close           1854 non-null   float64                         
 5   Volume          1854 non-null   int64                           
 6   Dividends       1854 non-null   float64                         
 7   Stock Splits    1854 non-null   float64                         
 8   Ticker          1854 non-null   object                          
 9   Capital Gains   309 non-null    float64                         
 10  win_loss_value  1854 non-null   float64         

<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-7-cf85b0d685d2>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


TypeError: ignored

Next Step = Cam to find Sectors and indexes